# Is there a Glass Ceiling in Wikipedia?

In this notebook we evaluate the presence of a glass ceiling (i.e., women need to be more notable than men to be able to appear on Wikipedia) by comparing notability between genders using statistical analysis (through the `statsmodels` package). Notability is defined as the number of editions a biography appears in. Additionally, we control by time and person class based on the DBpedia ontology.

By [Eduardo Graells-Garrido](http://carnby.github.io).

In [ ]:
from __future__ import division

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import powerlaw
import dbpedia_config

%matplotlib inline
sns.set(context='poster', font_scale=0.8, font='Source Code Pro', style='whitegrid')

In [ ]:
target_folder = dbpedia_config.TARGET_FOLDER

In [ ]:
data = pd.read_csv('{0}/consolidated_person_data.csv.gz'.format(target_folder), encoding='utf-8')
data.head()

In [ ]:
data.shape

In [ ]:
data.wikidata_entity.describe()

In [ ]:
data.available_english.value_counts()

In [ ]:
data.rename(columns={'class': 'dbpedia_class'}, inplace=True)

In [ ]:
data.dbpedia_class.value_counts()

In [ ]:
data['century'] = np.floor(data.birth_year / 100.0)

In [ ]:
data['birth_decade'] = np.floor(data.birth_year / 10.0)
data['birth_decade'].describe()

In [ ]:
data.century.describe()

In [ ]:
valid_data = data[~pd.isnull(data.birth_year) & (data.birth_year <= 2015) & (data.birth_year >= 0)]
valid_data.shape

In [ ]:
sample = valid_data

In [ ]:
sample.gender.value_counts() / sample.gender.value_counts().sum()

In [ ]:
sample.available_english.value_counts()

In [ ]:
sample.birth_year.describe()

In [ ]:
sample = valid_data
split_year = 1900
pre_group = sample[sample.birth_year < split_year]
post_group = sample[sample.birth_year >= split_year]

In [ ]:
pre_group.shape, post_group.shape

In [ ]:
pre_group.gender.value_counts() / pre_group.shape[0]

In [ ]:
post_group.gender.value_counts() / post_group.shape[0]

In [ ]:
fit_pre_f = powerlaw.Fit(pre_group.edition_count[pre_group.gender == 'female'], fit_method='KS', discrete=True)
fit_post_f = powerlaw.Fit(post_group.edition_count[post_group.gender == 'female'], fit_method='KS', discrete=True)
fit_pre_f.power_law.alpha, 2.0 * fit_pre_f.power_law.sigma, fit_post_f.power_law.alpha, 2.0 * fit_post_f.power_law.sigma

In [ ]:
fit_pre_f.power_law.D, fit_post_f.power_law.D

In [ ]:
fit_pre_f.distribution_compare('power_law', 'exponential'), fit_post_f.distribution_compare('power_law', 'exponential') 

In [ ]:
fit_pre_m = powerlaw.Fit(pre_group.edition_count[pre_group.gender == 'male'], fit_method='KS', discrete=True)
fit_post_m = powerlaw.Fit(post_group.edition_count[post_group.gender == 'male'], fit_method='KS', discrete=True)
fit_pre_m.power_law.alpha, 2.0 * fit_pre_m.power_law.sigma, fit_post_m.power_law.alpha, 2.0 * fit_post_m.power_law.sigma

In [ ]:
fit_pre_m.power_law.D, fit_post_m.power_law.D

In [ ]:
fit_pre_m.distribution_compare('power_law', 'exponential'), fit_post_m.distribution_compare('power_law', 'exponential') 

In [ ]:
plt.figure(figsize=(16,6))

plt.subplot(121)

fit_pre_f.power_law.plot_pdf(label='Women (fit, $\\alpha = {0:.2f}$)'.format(fit_pre_f.power_law.alpha))
fit_pre_f.plot_pdf(linear_bins=True, linestyle='none', marker='.', label='Women')

fit_pre_m.power_law.plot_pdf(label='Men (fit, $\\alpha = {0:.2f}$)'.format(fit_pre_m.alpha))
fit_pre_m.plot_pdf(linear_bins=True, linestyle='none', marker='.', label='Men')

plt.legend()
plt.title('Before {0}'.format(split_year))
plt.xlabel('Edition Count')
plt.ylabel('$p(x)$')
plt.xlim([1, 150])

plt.subplot(122)

fit_post_f.power_law.plot_pdf(label='Women (fit, $\\alpha = {0:.2f}$)'.format(fit_post_f.power_law.alpha))
fit_post_f.plot_pdf(linear_bins=True, linestyle='none', marker='.', label='Women')

fit_post_m.power_law.plot_pdf(label='Men (fit, $\\alpha = {0:.2f}$)'.format(fit_post_m.power_law.alpha))
fit_post_m.plot_pdf(linear_bins=True, linestyle='none', marker='.', label='Men')

plt.legend()
plt.title('{0} -- Present'.format(split_year))
plt.xlabel('Edition Count')
plt.ylabel('$p(x)$')

plt.xlim([1, 150])

In [ ]:
plt.figure(figsize=(16,6))

plt.subplot(121)

fit_pre_f.plot_ccdf(linestyle='none', marker='.', label='Women')
fit_pre_m.plot_ccdf(linestyle='none', marker='.', label='Men')
plt.xlabel('Edition Count')
plt.ylabel('$p(x \geq X)$')
plt.legend()
plt.title('Before {0}'.format(split_year))

plt.subplot(122)

fit_post_f.plot_ccdf(linestyle='none', marker='.', label='Women')
fit_post_m.plot_ccdf(linestyle='none', marker='.', label='Men')
plt.xlabel('Edition Count')
plt.ylabel('$p(x \geq X)$')
plt.legend()
plt.title('{0} -- Present'.format(split_year))

In [ ]:
plt.figure(figsize=(16,6))

plt.subplot(121)

rank_m_pre = pre_group[pre_group.gender == 'male'].edition_count.sort(ascending=False, inplace=False)
rank_f_pre = pre_group[pre_group.gender == 'female'].edition_count.sort(ascending=False, inplace=False)

plt.plot(np.arange(rank_m_pre.shape[0], dtype=np.float) / rank_m_pre.shape[0] + 0.001, rank_m_pre, label='Men', linestyle='none', marker='.', alpha=0.5)
plt.plot(np.arange(rank_f_pre.shape[0], dtype=np.float) / rank_f_pre.shape[0] + 0.001, rank_f_pre, label='Women', linestyle='none', marker='.', alpha=0.5)
plt.xlabel('Normalized Rank')
plt.ylabel('# of Editions')
plt.yscale('log')
plt.xscale('log')
plt.legend()
plt.title('Pre {0}'.format(split_year))
plt.xlim([0.001, 1.001])

plt.subplot(122)

rank_m_post = post_group[post_group.gender == 'male'].edition_count.sort(ascending=False, inplace=False)
rank_f_post = post_group[post_group.gender == 'female'].edition_count.sort(ascending=False, inplace=False)

plt.plot(np.arange(rank_m_post.shape[0], dtype=np.float) / rank_m_post.shape[0] + 0.001, rank_m_post, label='Men', linestyle='none', marker='.', alpha=0.75)
plt.plot(np.arange(rank_f_post.shape[0], dtype=np.float) / rank_f_post.shape[0] + 0.001, rank_f_post, label='Women', linestyle='none', marker='.', alpha=0.75)

plt.xlabel('Normalized Rank')
plt.ylabel('# of Editions')
plt.yscale('log')
plt.xscale('log')
plt.legend()
plt.title('{0} -- Present'.format(split_year))
plt.xlim([0.001, 1.001])

## Regressions

In [ ]:
from statsmodels.formula.api import negativebinomial

In [ ]:
m = negativebinomial("edition_count ~ C(gender,Treatment(reference='male')) + C(dbpedia_class,Treatment(reference='http://dbpedia.org/ontology/Person')) + birth_decade", 
        data=sample).fit_regularized()
m.summary()

In [ ]:
pre_group.shape[0] / sample.shape[0]

In [ ]:
pre_group.century.describe()

In [ ]:
m_pre = negativebinomial("edition_count ~ C(gender,Treatment(reference='male')) + C(dbpedia_class,Treatment(reference='http://dbpedia.org/ontology/Person')) + birth_decade", 
        data=pre_group, missing='raise').fit_regularized()
m_pre.summary()

In [ ]:
m_post = negativebinomial("edition_count ~ C(gender,Treatment(reference='male')) + C(dbpedia_class,Treatment(reference='http://dbpedia.org/ontology/Person')) + birth_decade", 
        data=post_group).fit_regularized()
m_post.summary()

In [ ]:
#summary = m_post.summary()
#print summary.as_latex().replace('http://dbpedia.org/ontology/', '').replace('_', '\_').replace(
#    ", Treatment(reference='Person')", '').replace(", Treatment(reference='male')", '').replace(
#    '\\textbf{', '').replace('}', '').replace('0.000', '$< 0.001$')

In [ ]:
pd.set_option('max_colwidth', 1000)

In [ ]:
table = pd.concat([m_pre.params.to_frame(), m_pre.bse.to_frame(), m_pre.pvalues, 
                   m_post.params.to_frame(), m_post.bse.to_frame(), m_post.pvalues,
                   m.params.to_frame(), m.bse.to_frame(), m.pvalues], axis=1, ignore_index=False)
print table.to_latex(float_format=lambda x: '{0:.3f}'.format(x)).replace('http://dbpedia.org/ontology/', '').replace('_', '\_').replace(
    ", Treatment(reference='Person')", '').replace(", Treatment(reference='male')", '').replace(
    '\\textbf{', '').replace('}', '').replace('0.000', '$^{***}$').replace('dbpedia\\\\_class', 'class').replace('nan', '--')

In [ ]:
table

In [ ]:
pd.concat([np.exp(m_pre.params.to_frame()), np.exp(m_post.params.to_frame()), np.exp(m.params.to_frame())], 
          axis=1, ignore_index=True)

In [ ]:
def deviance(m):
    return m.bic + m.df_resid * np.log(m.nobs)

print('AIC & \multicolumn{{3}}{{c|}}{{{0:.3f}}}  & \multicolumn{{3}}{{c|}}{{{1:.3f}}} & \multicolumn{{3}}{{c|}}{{{2:.3f}}}'.format(m_pre.aic, m_post.aic, m.aic))
print('BIC & \multicolumn{{3}}{{c|}}{{{0:.3f}}}  & \multicolumn{{3}}{{c|}}{{{1:.3f}}} & \multicolumn{{3}}{{c|}}{{{2:.3f}}}'.format(m_pre.bic, m_post.bic, m.bic))
print('Log-Likelihood & \multicolumn{{3}}{{c|}}{{{0:.3f}}}  & \multicolumn{{3}}{{c|}}{{{1:.3f}}} & \multicolumn{{3}}{{c|}}{{{2:.3f}}}'.format(m_pre.llf, m_post.llf, m.llf))
print('Deviance & \multicolumn{{3}}{{c|}}{{{0:.3f}}}  & \multicolumn{{3}}{{c|}}{{{1:.3f}}} & \multicolumn{{3}}{{c|}}{{{2:.3f}}}'.format(deviance(m_pre), deviance(m_post), deviance(m)))
print('LL Ratio & \multicolumn{{3}}{{c|}}{{{0:.3f}}}  & \multicolumn{{3}}{{c|}}{{{1:.3f}}} & \multicolumn{{3}}{{c|}}{{{2:.3f}}}'.format(m_pre.llr, m_post.llr, m.llr))
print('N & \multicolumn{{3}}{{c|}}{{{0:.3f}}}  & \multicolumn{{3}}{{c|}}{{{1:.3f}}} & \multicolumn{{3}}{{c|}}{{{2:.3f}}}'.format(m_pre.nobs, m_post.nobs, m.nobs))